In [1]:
import sys
import cuentitas
from auxiliares import generadorHH, generadorFran  #generador, generador1E, generadorEmisividad, 

from numpy import pi, sqrt, sin, cos, exp, log10, array, real, conj
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy import interpolate
import seaborn as sns
import pandas as pd
import glob
import theano.tensor as TT
import random

import time

import pymc3 as pm

sns.set() #seaborn como default

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


In [ ]:
# A partir de generadorFran, que devuelve "len(anguloR)" valores de s0 y una emi en nadir, para los parametros
# elejidos, los convierto a forma tensorial y los devuelte en subtensores

def toFitHH(ep1,ep2,d,s1,l1,s2,l2,anguloR,landa): #Me devuelve un tensor de nn+mm, con nn valores de angulos s0 y mm emis
    if (ep1 != ep2):
        s0s = generadorHH(ep1,ep2,d,s1,l1,s2,l2,incAng,landa)
    
        pp = len(s0s[1])
    
        a = TT.zeros(pp)
        for i in range(pp):
            a = TT.set_subtensor(a[i], s0s[1][i])
    
    return a

In [ ]:
# Resuelve mediante inferencia bayesiana utilizando algoritmo sequential monte carlo sampler (SMC) los parametros ep1,ep2,etc
# que mejor resuelven los valores de entrada. 
# La devolucion es una matriz con la distribución posterior. Este este contiene para cada columna los juegos de parametros que 
# explican los valores de entrada a partir de un cierto error sigma definido

# sector: Valores de entrada de s0 y emi de la zona estudiada, definida por "auxiliares.py/medicion()" o por una funcion que 
#         devuelta en primer lugar un array de n angulos en donde se mide y en segundo lugar un array con las mediciones. 
# angWR: es el angulo sin ruido para simulaciones con emisividad, es indistinto el valor 
#         que tome si uso la f() sin emisividad
# landa: frecuencia de operacion
# samples: cantidad de muestras del espacio de parametros
# sigmas: array error de [deltaSo, deltaEmi]


def modelo(sector, angWR, landa, samples, sigmas): 
    
    phi = np.pi
    
    print('Valores de entrada: ', sector[1])
    print('Cantidad de datos: ', sector[1].shape)

    #Rango en donde estudiamos los parametros    
    ranges = {}
    ranges['ep1'] = 1.40001, 8 #corro ligeramente el punto inicial ep1 para que no diverja con ep2 en canal HH
    ranges['ep2'] = 1.4, 8
    ranges['d'] = 0.001, 0.5
    ranges['s1'] = 0.0004, 0.004
    ranges['s2'] = 0.0004, 0.004

    #distribucion a priori de cada uno
    with pm.Model() as model:
        ep1 = pm.Uniform('ep1', ranges['ep1'][0], ranges['ep1'][1])
        ep2 = pm.Uniform('ep2', ranges['ep2'][0], ranges['ep2'][1])
        d = pm.Uniform('d', ranges['d'][0], ranges['d'][1])
        s1 = pm.Uniform('s1', ranges['s1'][0], ranges['s1'][1])
        s2 = pm.Uniform('s2', ranges['s2'][0], ranges['s2'][1])

    #Funcion de costo a minimizar
    def f(ep1 = ep1,
          ep2 = ep2,
          d = d,
          s1 = s1,
          s2 = s2):

        return toFitHH(ep1,ep2,d,s1,0.015,s2,0.01,sector[0],landa)
    
    with model:
        function_pm = pm.Deterministic('s0f', f()) #funcion de costo
        observations = pm.Normal( "obs",  mu=real(function_pm), sigma=sigmas, observed=sector[1]) #observaciones
        trace = pm.sample_smc(samples, cores=4, parallel=True) #metodo de exploracion de parametros
    
    return trace

In [ ]:
# Angulos estudiados en datos sinteticos. Este array debe contar con al menos 1 valor.
# landa: longitudes de onda de operacion. Este array debe contar con al menos 1 valor.

# incAng = [15,20,25]
# landa = [0.025,0.05]

incAng = [20,60]
landa = [0.025]


# set1
generadorHH(1.93,5.0,0.3,0.001,0.015,0.002,0.01,incAng,landa)

In [ ]:
# Prueba de la forma tensorial para ajustar
qw = toFitHH(1.93,
            5.0,
            0.3,
            0.001,
            0.015,
            0.002,
            0.01,
            incAng,
            landa)
qw.eval()

In [ ]:
# Datos de entrada para la inferencia

# set1
surfaceModelParam = [1.93,
                    5.0,
                    0.3,
                    0.001,
                    0.002]

entradaModelo = generadorHH(surfaceModelParam[0], 
                surfaceModelParam[1],
                surfaceModelParam[2],
                surfaceModelParam[3],
                0.015,
                surfaceModelParam[4],                
                0.01,
                incAng,
                landa)
entradaModelo

In [ ]:
# m = 51
# X,Y,Wt = ig(m-1)

t0 = time.time()


muestras = 2
Nombre = 'Delta'

sig = np.asarray(len(incAng)*len(landa)*[0.1] + len(landa)*[0.01])

# sig = np.asarray(len(incAng)*[0.1])  #pruebas error

#set3
traza = modelo(generadorHH(1.93,
                              5.0,
                              0.3,
                              0.001,
                              0.015,
                              0.002,
                              0.01,
                              incAng,
                              landa),
                              np.asarray([10]),
                              landa,
                              muestras,
                              sig)
 
#------------ error para generadorEmisividad()) ------------#
# sig = np.asarray(len(angR)*[noiseSTD**2] + len(ang)*[0.001])
# traza = modelo([angR,seR],ang,muestras,sig)

In [ ]:
plotear = np.stack((traza['ep1'],traza['ep2'],traza['d'], traza['s1'], traza['s2'])).T
dfplotear = pd.DataFrame(plotear, columns = ['ep1','ep2','d','s1','s2'])

In [ ]:
modelInReduced = [1.93,
                  5.0,
                  0.3,
                  0.001,
                  0.002]

In [ ]:
f = sns.PairGrid(dfplotear)
# f.fig.suptitle(Nombre, y=1.01, fontsize = 'large')
f.map_upper(sns.histplot)

f.map_lower(sns.kdeplot, fill=True)
f.map_diag(sns.histplot, kde=True, color='.5')

for i in range(5):
    for j in range(5):
        if i == j:
            #ploteo el valor verdadero en las marginales
#             f.axes[j,i].axvline(modelInReduced[i], color='r', linestyle='dashed', linewidth=1)
            pass
        else:
            f.axes[j,i].scatter(modelInReduced[i], modelInReduced[j], marker="+", c='r', s=200) #color='k', linestyle='dashed', linewidth=1)
            
            
#         f.axes[j,i].xaxis.set_label_text(xlabels[i])
#         f.axes[j,i].yaxis.set_label_text(ylabels[j])
#         f.axes[j,i].xaxis.set_label_text(xlabels[i])
#         f.axes[j,i].scatter(1,1)



In [ ]:
t1 = time.time()
print("Tiempo de ejecución:", t1-t0, 'seg','(', (t1-t0)/3600,  'hs )')

nombre = "set1-2Capas-noNoise-2s0+1emi-20000-1lambda-6020-3"
# f.savefig('Resultados_HH/'+nombre+'.png', dpi=300)

In [ ]:
e1 = traza['ep1']
e2 = traza['ep2']
d = traza['d']
s1 = traza['s1']
s2 = traza['s2']

guardar = [e1,e2,d,s1,s2]

# f_ = np.savetxt('Resultados_HH/'+nombre+'Trazas.txt', guardar, delimiter=';')
# f2_ = np.savetxt('Resultados_HH/'+nombre+'Trazas-input.txt', np.asarray([np.asarray(incAng),np.asarray(modelInReduced)],dtype=object), fmt='%s', delimiter=';')